# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

# Inspection

In [412]:
import pandas as pd
import numpy as np

In [413]:
tictactoe = pd.read_csv('tic-tac-toe.csv')
tictactoe.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [414]:
tictactoe.describe

<bound method NDFrame.describe of     TL TM TR ML MM MR BL BM BR  class
0    x  x  x  x  o  o  x  o  o   True
1    x  x  x  x  o  o  o  x  o   True
2    x  x  x  x  o  o  o  o  x   True
3    x  x  x  x  o  o  o  b  b   True
4    x  x  x  x  o  o  b  o  b   True
..  .. .. .. .. .. .. .. .. ..    ...
953  o  x  x  x  o  o  o  x  x  False
954  o  x  o  x  x  o  x  o  x  False
955  o  x  o  x  o  x  x  o  x  False
956  o  x  o  o  x  x  x  o  x  False
957  o  o  x  x  x  o  o  x  x  False

[958 rows x 10 columns]>

In [415]:
tictactoe.isnull().sum()

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [416]:
tictactoe.columns

Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], dtype='object')

# Transfer into numeric values

In [417]:
from sklearn.preprocessing import LabelEncoder


num_tictactoe = preprocessing.LabelEncoder()
# num_tictactoe.fit([ 'TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR'])
# X[:,2] = num_tictactoe.transform(

tictactoe[tictactoe.select_dtypes(include=['object']).columns] = tictactoe.select_dtypes(include=['object']).apply(lambda col: num_tictactoe.fit_transform(col))

tictactoe.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,2,2,2,2,1,1,2,1,1,True
1,2,2,2,2,1,1,1,2,1,True
2,2,2,2,2,1,1,1,1,2,True
3,2,2,2,2,1,1,1,0,0,True
4,2,2,2,2,1,1,0,1,0,True


#### Split data into predictors and target

In [418]:
tictactoe_columns = tictactoe.columns

predictors = tictactoe[tictactoe_columns[tictactoe_columns != 'class']] # all columns except class
target = tictactoe['class'] # class column

In [419]:
target.head()

0    True
1    True
2    True
3    True
4    True
Name: class, dtype: bool

In [420]:
predictors.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,2,2,2,2,1,1,2,1,1
1,2,2,2,2,1,1,1,2,1
2,2,2,2,2,1,1,1,1,2
3,2,2,2,2,1,1,1,0,0
4,2,2,2,2,1,1,0,1,0


# Normalization the Input Data


In [421]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,1.002699,1.084387,1.002699,1.084387,-0.419857,-0.167231,1.002699,-0.167231,-0.286678
1,1.002699,1.084387,1.002699,1.084387,-0.419857,-0.167231,-0.286678,1.084387,-0.286678
2,1.002699,1.084387,1.002699,1.084387,-0.419857,-0.167231,-0.286678,-0.167231,1.002699
3,1.002699,1.084387,1.002699,1.084387,-0.419857,-0.167231,-0.286678,-1.418849,-1.576054
4,1.002699,1.084387,1.002699,1.084387,-0.419857,-0.167231,-1.576054,-0.167231,-1.576054


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

Caluclate the Inputs:

In [422]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

9

Split the Data into Training and Test:

In [423]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

# Library Imports 

In [424]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

## Build a Neural Network


Setting up a funtion:

In [425]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(150, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(500, activation='relu'))

    model.add(Dense(1000, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(500, activation='relu'))

    model.add(Dense(2, activation='softmax'))

    
    # compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

Train the Model:

In [426]:
# build the model
model = regression_model()
# import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# fit the model
model.fit(X_train, y_train, epochs=100, verbose=1)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: 'Sequential' object has no attribute 'to'

# Saving the model:

In [371]:
model.save("tic-tac-toe.keras")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

Load model:

In [372]:
from tensorflow.keras.models import load_model

# Load the Keras model
model = load_model("tic-tac-toe.keras")

Run prediction: 

In [373]:
# Make predictions on the test set
y_pred = model.predict(X_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [374]:
print(y_test.shape)  # Should be 1D: (num_samples,)
print(y_pred.shape) 

(288,)
(288, 2)


In [375]:
y_test.head()

836    False
477     True
350     True
891    False
855    False
Name: class, dtype: bool

In [376]:
y_train.head()

228     True
319     True
715    False
212     True
79      True
Name: class, dtype: bool

In [377]:
# import numpy as np

# # If y_pred is probabilities, get the predicted class (e.g., threshold=0.5 for binary classification)
# y_pred = (y_pred > 0.5).astype(int)  # For binary classification

In [378]:
y_pred

array([[9.99927759e-01, 7.21910474e-05],
       [1.41537289e-15, 1.00000000e+00],
       [6.59604638e-08, 9.99999881e-01],
       [9.99999762e-01, 2.09260349e-07],
       [1.00000000e+00, 1.57537719e-22],
       [7.62588911e-07, 9.99999285e-01],
       [6.96602552e-13, 1.00000000e+00],
       [1.86332429e-07, 9.99999762e-01],
       [9.62727331e-03, 9.90372837e-01],
       [4.83799964e-01, 5.16200066e-01],
       [4.15600783e-11, 1.00000000e+00],
       [6.81303886e-23, 1.00000000e+00],
       [8.15360466e-17, 1.00000000e+00],
       [9.99999881e-01, 1.24804316e-07],
       [8.61715300e-12, 1.00000000e+00],
       [1.34097365e-15, 1.00000000e+00],
       [1.67356789e-04, 9.99832690e-01],
       [9.97625744e-17, 1.00000000e+00],
       [9.99999881e-01, 1.55252920e-07],
       [9.99999881e-01, 9.35433064e-08],
       [1.00000000e+00, 3.84381566e-16],
       [1.02056667e-17, 1.00000000e+00],
       [1.00000000e+00, 3.44004611e-08],
       [5.64333302e-18, 1.00000000e+00],
       [9.999997

In [379]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.17999738454818726
Test accuracy: 0.9756944179534912


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

| Epochs      | Layers      | Neurons      | Accuracy      | loss funtion |
|---------------|---------------|---------------|---------------|---------------|
| 100 | 5 | 150/500/1000/500/1 |  0.9826388955116272 | 0.0312918983399868|
| 50 | 5 | 150/500/1000/500/1 | 0.9861111044883728 |0.0622735433280468|
| 50 | 5 | 500/1000/1000/500/1 |  0.9756944179534912 |0.07112326472997665|
| 500 | 5 | 150/500/1000/1000/500/1 | 0.9791666865348816 |0.19541221857070923|

**Which approach(es) did you find helpful to improve your model performance?**

In [380]:
# your answer here